# Проект 2 &mdash; Построение вопросно-ответной системы

## Царькова Анастасия, Скоробогатов Денис, Анвардинов Шариф

Необходимые библиотеки: gensim, pymorphy2, pandas, scikit-learn, torch, regex.

### 1. Поиск по документам

С русскоязычной документацией есть проблема, которая заключается в том, что по ней сложно найти вопросов. Можно, конечно, заплатить толокерам, но можно выбрать что-нибудь более популярное. Например, Википедию.

Для ответов на вопросы соберем порядка ~100к статей. У Wikimedia Foundation [есть дампы](https://dumps.wikimedia.org/), упорядоченные по времени создания; будем считать, что это примерно эквивалентно значимости тем.

In [1]:
!curl -LO https://dumps.wikimedia.org/ruwiki/20171201/ruwiki-20171201-pages-articles1.xml-p4p311181.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  334M  100  334M    0     0  1973k      0  0:02:53  0:02:53 --:--:-- 2015k


В gensim имеется скрипт, который преобразует статьи Википедии из XML в более удобный JSON-формат со схемой `[{"title": str, "section_titles": [str], "section_texts": [str]}]`, попутно разбивая по секциям.

In [3]:
!python -m gensim.scripts.segment_wiki -f ruwiki-20171201-pages-articles1.xml-p4p311181.bz2 > wiki.json

2017-12-20 17:21:34,470 : MainProcess : INFO : running /place/home/pyos/virtualenv/lib/python3.6/site-packages/gensim/scripts/segment_wiki.py -f ruwiki-20171201-pages-articles1.xml-p4p311181.bz2
2017-12-20 17:25:18,170 : MainProcess : INFO : finished running /place/home/pyos/virtualenv/lib/python3.6/site-packages/gensim/scripts/segment_wiki.py


Прежде чем выделять точный ответ, нужно найти небольшое подмножество документов, в котором он будет находиться. Для этого над списком статей поднимем простейший поисковый движок, использующий только три признака: BM25, важность статьи (которую мы приняли убывающей с ее номером) и пересечение запроса и заголовка.

In [5]:
!du -hs wiki.json

2.9G	wiki.json


Хоть задание и выполняется на сервере с 92 ГБ памяти, чтобы его можно было как-нибудь запустить где-то еще документы лучше читать с диска. Для этого нужен маппинг смещений документов в файле.

In [6]:
offsets = []
with open('wiki.json') as fd:
    c = 0
    for line in fd:
        offsets.append(c)
        c += len(line)

In [7]:
print(len(offsets))

97913


In [8]:
import json

def read_document(i):
    with open('wiki.json') as fd:
        fd.seek(offsets[i])
        return json.loads(fd.readline())

In [9]:
read_document(5)["title"]

'Соционика'

gensim вырезает не всю разметку, поэтому заранее подготовим пару функций для нормализации и токенизации статей.

In [10]:
def tokenize(text):
    return re.sub(r'[\W\d_]', ' ', text.replace('\u0301', '')).lower().split()

def strip_markup(text):
    return re.sub(r"['\[\]\*]|[^\S\n]", ' ', text.replace('\u0301', '')).strip()

Теперь нужно построить лемматизированный инвертированный индекс (`лемма => [(BM25 TF, документ)]`). 40 наиболее часто встречающихся слов будем считать стоп-словами и удалим из индекса.

In [11]:
import re
import heapq
import pymorphy2
import collections
import multiprocessing

def make_index_part(range):
    assert range.step is None or range.step == 1
    index = {}
    doclens = []
    morph = pymorphy2.MorphAnalyzer()
    with open('wiki.json') as fd:
        fd.seek(offsets[range.start])
        for line, i in zip(fd, range):
            a = json.loads(line)
            ws = tokenize(a["title"] + "\n\n" + "\n\n".join(a["section_texts"]))
            ws = collections.Counter(m[0].normal_form for w in ws for m in (morph.parse(w),) if m)
            n = sum(ws.values())
            for w, c in ws.items():
                index.setdefault(w, []).append((c, i))
            doclens.append(n)
    return index, doclens

def merge_index_parts(parts, k1=2, b=0.75):
    index = {}
    doclens = []
    for part, pdoclens in parts:
        for word, documents in part.items():
            index.setdefault(word, []).extend(documents)
        doclens.extend(pdoclens)
    avglen = sum(doclens) / len(doclens)
    for k, ds in index.items():
        ds[:] = [(tf * (k1 + 1) / (tf + k1 * (1 - b + b * doclens[docid] / avglen)), docid) for tf, docid in ds]
        ds.sort(reverse=True)
    for k in heapq.nlargest(40, index, key=lambda k: len(index[k])):
        del index[k]
    return index

def make_index(processes):
    n = len(offsets)
    k = int(n ** 0.5)
    ranges = [range(i, i + k) for i in range(0, n, k)]
    ranges.append(range(n - (n % k), n))
    pool = multiprocessing.Pool(processes=processes)
    try:
        return merge_index_parts(pool.imap(make_index_part, ranges))
    finally:
        pool.close()

<div style="padding:1em;background-color:#fff3dd">_CREATE_INDEX = False если не хочется терять полчаса. Файл с индексом: <a href="https://drive.google.com/open?id=1mnap90yWLdw1w1zcXDF7FkkmsYQ4PgGn">wiki.index</a> (569 MB)</div>

In [15]:
_CREATE_INDEX = True

In [16]:
%%time
import pickle

if _CREATE_INDEX:
    index = make_index(48)
    with open('wiki.index', 'wb') as fd:
        pickle.dump(index, fd)
else:
    with open('wiki.index', 'rb') as fd:
        index = pickle.load(fd)

print('Terms in index:', len(index))

Terms in index: 1116242
CPU times: user 7min 51s, sys: 15.1 s, total: 8min 6s
Wall time: 39min 53s


Собственно функция поиска будет просто выбирать документы с максимальным BM25 с двумя нормировочными коэффициентами чтобы учесть важность статей и их заголовки. Поскольку прочитать заголовок статьи довольно медленно, точная релевантность будет считаться только для топ 10n документов по приближенной (без учета заголовка).

In [32]:
import math

def title_boost(docs, terms, norm_terms):
    for relevance, docid in docs:
        otitle = read_document(docid)['title']
        title = tokenize(otitle) or ['']
        c1 = sum(w in terms or w in norm_terms for w in title) / len(title)
        c2 = sum(w in title or q in title for w, q in zip(terms, norm_terms)) / len(terms)
        yield relevance * (1 + c1 * c2), docid, otitle

def search(n, terms, k1=2, b=0.75):
    docs = {}
    morph = pymorphy2.MorphAnalyzer()
    norm_terms = [m[0].normal_form for t in terms for m in (morph.parse(t),) if m]
    for term in norm_terms:
        matched = index.get(term, [])
        if matched:
            idf = math.log(len(offsets)) - math.log(len(matched))
        for tf, docid in matched:
            docs[docid] = docs.get(docid, 0) + tf * idf / (docid / len(offsets) + 1)
    return heapq.nlargest(n, title_boost(heapq.nlargest(n * 10, ((v, k) for k, v in docs.items())), terms, norm_terms))

In [64]:
search(10, ['шонин', 'георгий'])

[(42.032896488939386, 4058, 'Шонин, Георгий Степанович'),
 (13.660760083490523, 25435, 'Первый отряд космонавтов СССР'),
 (10.610559852846151, 2564, '3 августа'),
 (9.09192417019814,
  13030,
  'Хронология пилотируемых космических полётов (1960-е)'),
 (9.086425316549203, 26161, 'Георгий'),
 (8.607579266306844, 3588, 'Гречко, Георгий Михайлович'),
 (8.510484015782772, 21819, 'Георгий Александрович'),
 (8.444983433795262, 11611, 'Гонгадзе, Георгий Русланович'),
 (8.30055957103546, 2489, '7 апреля'),
 (8.235992150989222, 97763, 'Мартынов, Георгий Сергеевич')]

In [34]:
search(10, ['россия'])

[(6.8192173664222704, 1, 'Россия'),
 (5.0439376004904162, 6874, 'Единая Россия'),
 (4.9291830861535288, 6655, 'Россия (фракция)'),
 (4.7466299055857846, 7726, 'ОМОН (Россия)'),
 (4.304848966103771, 12067, 'Отечество — Вся Россия'),
 (4.2628455761938611, 7365, 'Россия и Европейский союз'),
 (4.1404960573959393, 6653, 'Родина (партия, Россия)'),
 (4.1218742126260075, 16110, 'Живописная Россия (журнал)'),
 (3.8901328548299361, 6601, 'Наш дом — Россия (фракция)'),
 (3.5438126941157417, 340, 'День Конституции Российской Федерации')]

In [35]:
search(10, ['президент', 'литвы'])

[(21.667616662416695, 0, 'Литва'),
 (18.106700652127323, 16662, 'Президенты Литвы'),
 (16.115054462630866, 17660, 'Конституция Литвы'),
 (16.036375557681566, 17543, 'Президент Литовской Республики'),
 (15.117201428898323, 18971, 'Флаг Литвы'),
 (13.977364783870822, 17472, 'Бразаускас, Альгирдас Миколас'),
 (13.965689123542816, 18535, 'Паулаускас, Артурас'),
 (13.924998164732902, 18424, 'Паксас, Роландас'),
 (13.85146332590978, 17722, 'Сметона, Антанас'),
 (13.053533356185294, 22463, 'Срединная Литва')]

In [36]:
search(10, ['режиссер', 'корпорации', 'монстров'])

[(27.342255365209677, 42791, 'Корпорация монстров'),
 (14.269100819919847, 38486, 'Взвод монстров'),
 (13.439909988447205, 37771, 'Страшила (значения)'),
 (13.277872259581677, 97813, 'Doom 3: Resurrection of Evil'),
 (13.242000292664974, 40258, 'Монстр в коробке'),
 (12.978020438827361, 44569, 'Pixar'),
 (12.570208749196071, 18511, 'Корпорация'),
 (12.479924957909009, 17274, 'Наблюдатель'),
 (12.273718471169822, 38651, 'Молодой Франкенштейн'),
 (11.977213908102271, 17396, 'Фильм ужасов')]

In [37]:
search(10, ['расстояние', 'до', 'марса'])

[(22.776387011050335, 478, 'Марс'),
 (16.132226862021639, 556, 'Марс (значения)'),
 (16.055522207850881, 17820, 'Фобос'),
 (15.310275302363497, 29258, 'Маринер (космическая программа)'),
 (15.225510195386638, 17821, 'Деймос'),
 (14.859669632564481, 5270, 'Марс (мифология)'),
 (14.736046047194399, 389, 'XXI век'),
 (14.251467988783217, 2023, 'Солнечная система'),
 (14.168303607938626, 29586, 'Марс-экспресс'),
 (14.165355636124339, 4531, 'Астрономическая единица')]

Кажется, документы более-менее релевантные: топ &mdash; статьи о нужных нам объектах или людях, а дальше несколько связанных (и немного несвязанных) статей.

### 2. Поиск в параграфе

<div style="padding:1em;background-color:#fff3dd">Модель обучается <em>очень</em> долго. Результат: <a href="https://drive.google.com/open?id=1sI3ch3itKzwicwhP2Ydp0E5YzT5oIjEV">20171217-172fc9a0.mdl</a> (68 MB) надо поместить в поддиректорию models.</div>

Для второй части вопросно-ответной системы обучим на скачанном датасете Сбербанка модуль reader из модели [Facebook DrQA](https://github.com/facebookresearch/DrQA). Для модели нужен word2vec из проекта [RusVectores](http://rusvectores.org/ru/models/) DrQA принимает вектора в текстовом виде, так что конвертируем их gensim-ом.

In [38]:
!curl -LO http://rusvectores.org/static/models/ruwikiruscorpora_0_300_20.bin.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  421M  100  421M    0     0  7276k      0  0:00:59  0:00:59 --:--:-- 7315k


In [40]:
!mkdir -p data/embeddings

In [41]:
import gensim.models.keyedvectors

w2v = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format("ruwikiruscorpora_0_300_20.bin.gz", binary=True)
with open("data/embeddings/ruwiki_300.txt", "w") as outfile:
    for word in w2v.vocab:
        vec = w2v[word]
        word = word.replace("::", "_")
        word = word.split("_")[0]
        print(word, *vec, file=outfile)

Обучающая выборка для выделяющей ответы в параграфах модели: ~50к троек (параграф, вопрос, ответ) из задания B соревнования [Сбербанка Data Science Journey 2017](https://sdsj.ru/ru/). Вопросы и ответы придуманы пользователями на Толоке.

In [4]:
!curl -LO https://sdsj.ru/train_task_b.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 74.9M  100 74.9M    0     0  20.9M      0  0:00:03  0:00:03 --:--:-- 20.9M


~10000 векторов отделим в валидационную выборку.

In [42]:
!mkdir -p data/datasets

In [45]:
import pandas
import sklearn.utils

data = sklearn.utils.shuffle(pandas.read_csv('train_task_b.csv'), random_state=42)[['paragraph_id', 'question_id', 'paragraph', 'question', 'answer']]
data.iloc[:10000].to_csv("data/datasets/validate.csv", header=True, index=False)
data.iloc[10000:].to_csv("data/datasets/train.csv", header=True, index=False)

Теперь данные надо преобразовать из CSV в формат DrQA.

In [46]:
!PYTHONPATH=.:$PYTHONPATH python scripts/reader/preprocess.py --tokenizer SimpleTokenizer data/datasets/train.csv data/datasets/train-drqa.json

found few errors with this Tokenizer: 5
Total time: 120.5091 (s)


In [47]:
!PYTHONPATH=.:$PYTHONPATH python scripts/reader/preprocess.py --tokenizer SimpleTokenizer data/datasets/validate.csv data/datasets/validate-drqa.json

found few errors with this Tokenizer: 0
Total time: 43.0068 (s)


И, наконец, можно обучить модель (название при этом генерируется случайное):

In [60]:
!mkdir -p models

In [ ]:
!PYTHONPATH=.:$PYHONPATH python scripts/reader/train.py --checkpoint True --train-file train-drqa.json --dev-file validate-drqa.json --embedding-file ruwiki_300.txt --use-lemma True --official-eval False --batch-size 60 --expand-dictionary False --uncased-doc True --uncased-question True --restrict-vocab True --valid-metric exact_match --doc-layers 2 --question-layers 2 --hidden-size 32 >out.log 2>err.log

### 3. Проверка результатов

In [48]:
import drqa.reader

In [50]:
predictor = drqa.reader.Predictor('models/20171217-172fc9a0.mdl')

Namespace(concat_rnn_layers=True, doc_layers=3, dropout_emb=0.4, dropout_rnn=0.4, dropout_rnn_output=True, embedding_dim=300, fix_embeddings=True, grad_clipping=10, hidden_size=128, learning_rate=0.1, max_len=15, model_type='rnn', momentum=0, num_features=4, optimizer='adamax', question_layers=3, question_merge='self_attn', rnn_padding=False, rnn_type='lstm', tune_partial=0, use_in_question=True, use_lemma=True, use_ner=False, use_pos=False, use_qemb=True, use_tf=True, vocab_size=44249, weight_decay=0)


Будем искать ответ в каждом параграфе релевантных статей, считая что отрывки из более релевантных статей с большей вероятностью отвечают на вопрос. Для поискового запроса возьмем сам вопрос без вопросных маркеров.

In [72]:
def answer(question, stopwords={'кто', 'где', 'когда', 'зачем', 'почему', 'сколько', 'каково'}, top_n=3):
    terms = [t for t in tokenize(question) if t not in stopwords]
    answers = []
    for relevance, docid, title in search(10, terms):
        for section in read_document(docid)['section_texts']:
            for paragraph in section.split('\n\n'):
                paragraph = strip_markup(paragraph)
                if len(paragraph.split()) > 10:
                    for answer, score in predictor.predict(paragraph, question, top_n=top_n * 5):
                        answers.append((score * relevance ** 0.5, answer, paragraph))
    answers.sort(reverse=True)
    return answers[:top_n]

In [52]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [73]:
%%time
answer('Где жил Пушкин?', top_n=3)

CPU times: user 12min 21s, sys: 1.75 s, total: 12min 22s
Wall time: 1min 1s


[(3.0233459882036513,
  'в спальном районе Брюсселя.',
  'Биография А.А.Пушкина.\n  \n   Пушкин и Армия России.\n   Последний Пушкин живет в спальном районе Брюсселя.'),
 (2.1925385083262925,
  'в июне 1825 года,',
  'Следующая встреча Анны Керн с Пушкиным случилась в июне 1825 года, когда она приехала в Тригорское.\nИменно там Пушкин написал Керн знаменитое стихотворение-мадригал "К   "(«Я помню чудное мгновенье…»). В то же время, Анна увлеклась приятелем поэта (и сыном Осиповой, своим двоюродным братом) Алексеем Вульфом.\nВпрочем, кокетничала она и с соседом Вульфов — помещиком Рокотовым.'),
 (1.9553251271624139,
  '7 января 1833 года Пушкин',
  '7 января 1833 года Пушкин был избран членом Российской академии одновременно с П. А. Катениным, М. Н. Загоскиным, Д. И. Языковым и А. И. Маловым.')]

..?

In [74]:
%%time
answer('Когда была основана Россия?', top_n=3)

CPU times: user 15min 32s, sys: 2.26 s, total: 15min 34s
Wall time: 1min 16s


[(2.4381311220828197,
  'в начале XVIII века,',
  '=== Печать ===\nПервые периодические издания появились в России в начале XVIII века, однако широкое развитие печатные средства массовой информации получили только в конце XIX века. При этом, если для конца XIX — начала XX века была характерна относительная свобода печати, то для периода СССР — более жёсткая политическая цензура и более высокая степень государственного контроля над печатью. Радикальные подвижки в плане обеспечения свободы печати произошли в ходе демократических реформ конца 1980-х годов. В этот период имел место существенный рост количества периодических изданий, достаточно чётко обозначилась принадлежность тех или иных газет и журналов к различным политическим и общественным течениям.'),
 (1.9513096059890718,
  'Номинальная средняя зарплата работника в',
  'Номинальная средняя зарплата работника в России по итогам января 2016 года составила 32 122 рубля в месяц.'),
 (1.8873475109863831,
  'Динамика индекса восприятия к

..???

In [75]:
%%time
answer('Кто президент Литвы?', top_n=3)

CPU times: user 5min, sys: 780 ms, total: 5min 1s
Wall time: 24.2 s


[(3.4636813966416691,
  '25 октября 1992 года.',
  'Действующая в настоящее время Конституция Литовской Республики была принята всенародным референдумом 25 октября 1992 года.'),
 (3.2983522113980444,
  'В XVI—XVIII веках в',
  'В XVI—XVIII веках в Литве по польскому образцу сложилась политическая система, известная как шляхетская демократия. Она характеризовалась наличием широких прав шляхты (дворянства) в управлении государством. Одновременно с этим происходила полонизация шляхты, выраженная в перенимании правящим сословием Великого княжества Литовского польского языка, культуры и идентичности. На непривилегированные сословия полонизация столь значительного влияния не оказала.'),
 (3.2765870438033429,
  'В июне 2008 года парламент',
  '=== Внутренняя политика ===\nВ июне 2008 года парламент Литвы принял закон, уравнивающий нацистскую и советскую символику и запрещающий её использование в публичных местах: она «  может восприниматься как пропаганда нацистских и коммунистических оккупац

..??????

In [76]:
%%time
answer('Кто режиссер "Корпорации Монстров"?', top_n=3)

CPU times: user 5min 1s, sys: 688 ms, total: 5min 2s
Wall time: 24.2 s


[(2.136749748685336,
  'Фредди Крюгер\n Маньяк',
  'Фредди Крюгер\n Маньяк из серии фильмов «Кошмар на улице Вязов», которого сожгли при жизни и который стал приходить к детям во снах. Смерть во сне вела за собой смерть в реальности.'),
 (2.0488214819676305,
  'Джон Крамер\nМаньяк',
  'Джон Крамер\nМаньяк из серии «Пила», имеет прозвище «конструктор», он одержим идеей научить людей ценить жизнь, сам лично он своих жертв не убивает, а подстраивает так, чтобы жертвы сами себя убивали.'),
 (1.9233563672181768,
  'Альфред Хичкок,',
  'Альфред Хичкок, большинство работ которого больше относятся к триллерам, чем собственно к фильмами ужасов, снимает в этот период знаковый для жанра фильм «Психо», в котором сюжет сосредоточен на маньяке-убийце, а не на его жертвах и попытках поймать преступника, а сам фильм развивает тему психологических триллеров о «женщинах в опасности».')]

..?????????

In [78]:
%%time
answer('Каково расстояние до Луны?', top_n=3)

CPU times: user 2min 48s, sys: 492 ms, total: 2min 48s
Wall time: 13.5 s


[(3.3097845227422815,
  '75 часов.',
  '3 февраля 1966 года впервые в истории освоения космоса совершила мягкую посадку на поверхность Луны и впервые передала на Землю телепанорамы лунной поверхности. Продолжительность активного существования автоматической лунной станции (АЛС) на поверхности Луны составила 75 часов.'),
 (3.3023328046779503,
  '6200 км от',
  'Файл:Luna3-rus.svg|thumb|left|400px|Траектория «Луны-3» и гравитационный манёвр\nПосле старта с космодрома Байконур космический аппарат «Луна-3» вышел на сильно вытянутую эллиптическую орбиту искусственного спутника Земли с наклонением 75° и периодом обращения 22 300 мин и обогнул обратную сторону Луны по направлению с юга на север, пройдя на расстоянии 6200 км от её поверхности. Под действием гравитации Луны орбита аппарата изменилась; кроме того, поскольку Луна продолжала двигаться по своей орбите, изменилась и плоскость орбиты космического аппарата. Изменение орбиты было рассчитано так, чтобы аппарат при возвращении к Земле сн

В общем, Пушкин жил в спальном районе Брюсселя, Россия была основана в начале 18 века, режиссером "Корпорации Монстров" был Фредди Крюгер, а до Луны 75 часов. Отличная система. От того что ожидалось.